In [1]:
import ee
import geemap
import os


In [2]:
# geemap.update_package()

In [3]:
Map = geemap.Map()


In [4]:
# mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
# clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
# landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
# landsat_vis = {
#     'min': 0,
#     'max': 1,
#     'bands': ['b5', 'b4', 'b3']
# }
# Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
# Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
# Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
# Map

In [5]:
countries = ee.FeatureCollection('users/giswqs/public/countries')
Map.addLayer(countries, {}, 'countries')


In [6]:
roi = countries.filter(ee.Filter.eq('id', 'CHL'))
Map.addLayer(roi, {}, 'roi')

In [7]:
region = ee.FeatureCollection('users/franciscoalbornoz/Aysen')
Map.addLayer(region, {}, 'Aysen')

In [8]:
start_date = '2018-12-01'
end_date = '2019-03-31'

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date)

In [9]:
median = l8.median()

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
}

Map.addLayer(median, visParams, 'Median')
Map.centerObject(roi, 10)
Map

Map(center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['position'], widget=HBox…

In [10]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [11]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [12]:
def addMonth(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('M'))
    return image.addBands(ee.Image(img_doy).rename('month').toInt())

In [13]:
def addDOY(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('D'))
    return image.addBands(ee.Image(img_doy).rename('doy').toInt())

In [14]:
withNDVI = l8.map(addNDVI).map(addDate).map(addMonth).map(addDOY)

In [15]:
greenest = withNDVI.qualityMosaic('NDVI').clip(region)

In [16]:
greenest.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'date',
 'month',
 'doy']

In [17]:
ndvi = greenest.select('NDVI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map.addLayer(ndvi, {'palette': palette}, 'NDVI')

In [18]:
Map.addLayer(greenest, visParams, 'Greenest pixel')
Map

Map(center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['position'], widget=HBox…

In [19]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
Map

Map(center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['position'], widget=HBox…

In [20]:
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]
# paleta de 18 colores


In [21]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]

In [22]:
landcover_2018.bandNames().getInfo()

['b1']

In [23]:
clasificacion_2018.bandTypes().getInfo()


{'b1': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}}

In [24]:
# land2018 = landcover_2018.set('classification_class_values', class_values)
# land2018f = land2018.set('classification_class_palette', class_palette)

In [25]:
# class2018 = clasificacion_2018.set('classification_class_values', class_values)
# class2018f = class2018.set('classification_class_palette', class_palette)

In [26]:
# Map.addLayer(land2018f, {}, 'landcover_prov_coyhaique' )
# Map.addLayer(class2018f,{},'clasificacion 2018')

In [27]:
mosaico.bandNames().getInfo()

['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

In [28]:
work_dir = os.path.expanduser('../Data/shapes/')

val_shp= os.path.join(work_dir, 'validacion.shp')
merged_shp = os.path.join(work_dir, 'roi_merged.shp')

In [29]:
merged =geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [30]:
val = geemap.shp_to_ee(val_shp)
Map.addLayer(val, {}, 'Áreas de Validación')


In [31]:
merged_shp

'../Data/shapes/roi_merged.shp'

In [32]:
dataset = greenest.sampleRegions(**{
    'collection': merged,
    'properties': ['ID'],
    'scale' : 30 ,
})

In [33]:
datasetval = greenest.sampleRegions(**{
    'collection': val,
    'properties': ['ID'],
    'scale': 10,
})

In [34]:
dataset.size().getInfo()

20322

In [35]:
datasetval.size().getInfo()

4679

In [36]:
print(dataset.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 0.10622850805521011, 'B10': 277.4976806640625, 'B11': 277.0090026855469, 'B2': 0.08158686012029648, 'B3': 0.05989211052656174, 'B4': 0.03704666346311569, 'B5': 0.3357214033603668, 'B6': 0.08223237097263336, 'B7': 0.02972152642905712, 'B8': 0.04830099269747734, 'B9': 0.0024417066015303135, 'BQA': 2720, 'ID': 1, 'NDVI': 0.801234781742096, 'date': 20190210, 'doy': 41, 'month': 2}}


In [37]:
# Guardamos las bandas
bands = ['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'date',
 'month',
 'doy']

# Esta propiedad guarda las clases de landcover
label = 'ID'

# Overlay the points on the imagery to get training.
# sample = mosaico.select(bands).sampleRegions(**{
#   'collection': dataset,
#   'properties': ['RF'],
#   'scale': 30
# })

# Adds a column of deterministic pseudorandom numbers. 
# sample = dataset.randomColumn()

# split = 0.7 

# training = sample.filter(ee.Filter.lt('random', split))
# validation = sample.filter(ee.Filter.gte('random', split))

In [38]:
# print(training.first().getInfo())

In [39]:
# print(validation.first().getInfo())

In [40]:
randomforest = ee.Classifier.smileRandomForest(100).train(dataset, label, bands)
resultRF = greenest.select(bands).classify(randomforest)

In [41]:
SVM = ee.Classifier.libsvm('Voting','C_SVC','RBF', gamma = 0.5 , cost = 10 )
trained = SVM.train(dataset, label, bands)
classifiedSVM = greenest.classify(trained)

In [42]:
smc = ee.Classifier.smileCart().train(dataset, label, bands)
classifiedsmc = greenest.select(bands).classify(smc)

In [43]:
# Clasificando con las mismas bandas de entrenamiento
# result1 = greenest.select(bands).classify(randomforest)
# result2 = greenest.select(bands).classify(SVM)
# # Display the clusters with random colors.
# Map.addLayer(result.randomVisualizer(), {}, 'classificado')
# Map

In [44]:
# result.getInfo()

In [45]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]

In [46]:
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]

In [47]:
landcover_randomforest = resultRF.set('classification_class_values', class_values)
landcover_randomforest = landcover_randomforest.set('classification_class_palette', class_palette)

In [48]:
Map.addLayer(landcover_randomforest, {}, 'Land cover Random Forest Región')

In [50]:
landcover_svm = classifiedSVM.set('classification_class_values', class_values)
landcover_svm = landcover_svm.set('classification_class_palette', class_palette)

In [51]:
Map.addLayer(landcover_svm, {}, 'Land cover SVM Región')

In [52]:
landcover_cart = classifiedsmc.set('classification_class_values', class_values)
landcover_cart = landcover_cart.set('classification_class_palette', class_palette)

In [53]:
Map.addLayer(landcover_cart, {}, 'Land cover SCART Región')

In [54]:
Map

Map(bottom=160691.0, center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['positi…

RF

In [55]:
train_accuracy = randomforest.confusionMatrix()

In [56]:
train_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 890, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1119, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 4319, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 2302, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 139, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1331, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 881, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 73, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 120, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 320, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 232, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4473, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 366, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1107, 0, 0, 0, 0],
 [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 306, 0, 0, 0],
 [0, 

In [57]:
train_accuracy.accuracy().getInfo()

0.9998031689794311

In [58]:
train_accuracy.kappa().getInfo()

0.9997744803922869

In [59]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0.9864864864864865],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0.9990974729241877],
 [0.9935064935064936],
 [1],
 [1],
 [1]]

In [60]:
train_accuracy.consumersAccuracy().getInfo()

[[0,
  0.9988776655443322,
  1,
  0.999537144179588,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0.9953917050691244,
  1]]

In [61]:
validacion = datasetval.classify(randomforest)

In [62]:
validacion.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'B1': 0.1297835111618042,
  'B10': 278.9761047363281,
  'B11': 276.3595275878906,
  'B2': 0.10309768468141556,
  'B3': 0.06259501725435257,
  'B4': 0.046153344213962555,
  'B5': 0.0707976296544075,
  'B6': 0.03525298461318016,
  'B7': 0.027852408587932587,
  'B8': 0.05712661147117615,
  'B9': 0.0011301300255581737,
  'BQA': 2976,
  'ID': 3,
  'NDVI': 0.21072322130203247,
  'classification': 3,
  'date': 20190321,
  'doy': 80,
  'month': 3}}

In [63]:
testAccuracy = validacion.errorMatrix('ID', 'classification')

In [64]:
testAccuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 46, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 14],
 [0, 0, 0, 41, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 16, 0, 0, 0],
 [0, 0, 0, 357, 472, 0, 0, 3, 8, 0, 0, 0, 0, 10, 0, 35, 0, 0, 0],
 [0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 37, 4, 101, 0, 0, 245, 0, 0, 36, 0, 0, 78, 0, 0, 0, 0, 84],
 [0, 0, 0, 0, 0, 4, 2, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 2, 55, 0, 1, 1, 17, 19, 0, 0, 20, 9, 28, 22, 4, 0, 0, 0, 29],
 [0, 10, 40, 0, 0, 3, 117, 0, 0, 0, 0, 73, 8, 15, 64, 0, 30, 4, 13],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 40, 0, 10, 0, 0, 0, 0],
 [0, 2, 157, 0, 4, 10, 11, 4, 0, 0, 5, 4, 3, 63, 0, 0, 0, 0, 160],
 [0, 612, 8, 0, 10, 2, 243, 0, 0, 0, 0, 8, 41, 0, 494, 0, 130, 10, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [65]:
testAccuracy.accuracy().getInfo()

0.37337037828595854

In [66]:
testAccuracy.kappa().getInfo()

0.30936372365704823

In [67]:
testAccuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.7540983606557377],
 [0.6212121212121212],
 [0.5333333333333333],
 [1],
 [1],
 [0.4188034188034188],
 [0.08333333333333333],
 [0.6666666666666666],
 [0.0966183574879227],
 [0.19363395225464192],
 [0.8],
 [0.14893617021276595],
 [0.3170731707317073],
 [1],
 [0.6014851485148515],
 [0],
 [0]]

In [68]:
testAccuracy.consumersAccuracy().getInfo()

[[0,
  0.0030165912518853697,
  0.13142857142857142,
  0.09382151029748284,
  0.7946127946127947,
  0.13043478260869565,
  0.0022522522522522522,
  0.9040590405904059,
  0.1111111111111111,
  1,
  0.32786885245901637,
  0.6822429906542056,
  0.32786885245901637,
  0.3351063829787234,
  0.7866242038216561,
  0.019230769230769232,
  0.6029776674937966,
  0,
  0]]

SVM

In [69]:
train_accuracy1 = trained.confusionMatrix()

In [70]:
train_accuracy1.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 633, 11, 0, 0, 0, 58, 0, 0, 0, 0, 0, 97, 0, 18, 0, 69, 0, 4],
 [0, 6, 608, 0, 0, 0, 1, 0, 0, 4, 0, 0, 1, 1, 0, 0, 37, 0, 461],
 [0, 0, 0, 4297, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0],
 [0, 0, 0, 64, 2160, 1, 0, 56, 0, 0, 2, 0, 0, 9, 0, 9, 0, 1, 0],
 [0, 0, 0, 0, 0, 52, 5, 0, 0, 0, 4, 15, 60, 0, 2, 0, 1, 0, 0],
 [0, 36, 2, 0, 0, 0, 651, 0, 0, 0, 0, 3, 507, 0, 26, 0, 105, 1, 0],
 [0, 0, 0, 0, 77, 0, 0, 804, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 2, 2, 0, 0, 0, 70, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 3, 0, 0, 0, 0, 0, 0, 116, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 13, 3, 0, 1, 0, 0, 297, 0, 0, 0, 0, 0, 0, 0, 6],
 [0, 0, 7, 0, 0, 11, 3, 0, 0, 5, 5, 145, 38, 1, 2, 0, 1, 0, 14],
 [0, 11, 3, 0, 0, 0, 136, 7, 0, 0, 0, 0, 4282, 0, 31, 0, 3, 0, 0],
 [0, 0, 0, 0, 4, 6, 0, 0, 0, 3, 4, 2, 10, 259, 0, 0, 0, 0, 78],
 [0, 62, 0, 0, 0, 0, 109, 0, 0, 0, 0, 0, 110, 0, 710, 0, 116, 1, 0],
 [0, 0, 0, 73, 12, 1, 0, 0, 0, 0, 

In [71]:
train_accuracy1.accuracy().getInfo()

0.8477019978348588

In [72]:
train_accuracy1.kappa().getInfo()

0.8239095239853439

In [73]:
train_accuracy1.producersAccuracy().getInfo()

[[0],
 [0.7112359550561798],
 [0.5433422698838248],
 [0.9949062282935864],
 [0.9383145091225021],
 [0.37410071942446044],
 [0.48910593538692715],
 [0.9125993189557321],
 [0.9459459459459459],
 [0.9666666666666667],
 [0.928125],
 [0.625],
 [0.9572993516655488],
 [0.7076502732240437],
 [0.6407942238267148],
 [0.7142857142857143],
 [0.7288842544316997],
 [0.6759259259259259],
 [0.9253218884120171]]

In [74]:
train_accuracy1.consumersAccuracy().getInfo()

[[0,
  0.7922403003754693,
  0.8456189151599444,
  0.9673570463755066,
  0.9452954048140044,
  0.7027027027027027,
  0.6072761194029851,
  0.9262672811059908,
  1,
  0.90625,
  0.9488817891373802,
  0.8734939759036144,
  0.8157744332253762,
  0.9487179487179487,
  0.885286783042394,
  0.9361702127659575,
  0.6753623188405797,
  0.9798657718120806,
  0.6561168594035302]]

In [76]:
validacion1 = datasetval.classify(trained)

In [77]:
testAccuracy1 = validacion1.errorMatrix('ID', 'classification')

In [78]:
testAccuracy1.accuracy().getInfo()

0.1837999572558239

In [79]:
testAccuracy1.accuracy().getInfo()

0.1837999572558239

In [ ]:
testAccuracy1.kappa().getInfo()

In [ ]:
testAccuracy1.producersAccuracy().getInfo()

In [ ]:
testAccuracy1.consumersAccuracy().getInfo()

SCART

In [ ]:
train_accuracy = smc.confusionMatrix()

In [ ]:
train_accuracy.getInfo()

In [ ]:
train_accuracy.accuracy().getInfo()

In [ ]:
train_accuracy.kappa().getInfo()

In [ ]:
train_accuracy.producersAccuracy().getInfo()

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

In [ ]:
validated = datasetval.classify(smc)

In [ ]:
validated.first().getInfo()

In [ ]:
test_accuracy = validated.errorMatrix('ID', 'classification')

In [ ]:
test_accuracy.getInfo()

In [ ]:
test_accuracy.accuracy().getInfo()

In [ ]:
test_accuracy.kappa().getInfo()

In [ ]:
test_accuracy.producersAccuracy().getInfo()

In [ ]:
test_accuracy.consumersAccuracy().getInfo()